<a href="https://colab.research.google.com/github/Santiago-R/aprende/blob/main/lm-hackers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [A hacker's guide to Language Models](https://colab.research.google.com/github/fastai/lm-hackers/blob/main/lm-hackers.ipynb#scrollTo=0b017bfc-5be0-4e41-9fa1-9f685c3b0de5)


## Initial restart

In [1]:
import os
import importlib
a_spec = importlib.util.find_spec('accelerate')
b_spec = importlib.util.find_spec('bitsandbytes')
found = a_spec is not None and b_spec is not None

if found: print('Dependencies installed in previous run ✔️')
else:
    !pip install accelerate -qq
    !pip install bitsandbytes -qq
    !pip install auto-gptq -qq
    !pip install optimum -qq
    !pip install vllm

    print('\nDependency installation requires restart --> killing runtime 💀')

Dependencies installed in previous run ✔️


In [2]:
if not found:
    os.kill(os.getpid(), 9)  # Kill runtime if required

## Setup

In [19]:
from google.colab import drive
drive.mount('/content/drive')  # , force_remount=True)

Mounted at /content/drive


In [20]:
# import tokenize
# from io import BytesIO

In [21]:
# Load api keys as environvent variable (from Drive's api_keys.env)
!pip install python-dotenv -qq
from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/LLM/api_keys.env')

True

## OpenAI API with system prompt

In [53]:
!pip install openai -qq
import openai
client = openai.OpenAI()
ChatCompletion = client.chat.completions
Completion = client.completions

#### Simple chat

In [130]:
from IPython.display import display, Markdown  # , HTML

In [131]:
class NewChat:
    def __init__(self, p_sys=None, model="gpt-4o-mini"):
        self.messages = [{"role": "system", "content": p_sys}] if p_sys else []
        self.model = model
    def __call__(self, p):
        self.messages.append({"role": "user", "content": p})
        answer = ChatCompletion.create(
            model=self.model,
            messages=self.messages
        ).choices[0].message.content
        self.messages.append({"role": "assistant", "content": answer})
        return answer
    def retry(self, index=-2):
        assert (user_input:=self.messages[index])["role"] == "user", "can only retry answering a user request"
        self.messages = self.messages[:index]
        return self(user_input["content"])
    def list_repr(self): return [f'###{m["role"].upper()}\n{m["content"]}' for m in self.messages]
    def __repr__(self, sep='\n'): return sep.join(self.list_repr())
    def display(self): display(Markdown(self.__repr__(sep='\n***\n')))

In [132]:
chat = NewChat(model="gpt-4o-mini")

In [133]:
answer = chat("""
I'd like to Make "use" HTML tags that are sub-elements of id "PathCollection_1" a bit bigger on hover. Brief response, please.
""")
print(answer)

To make the `<use>` elements within the element with the id `PathCollection_1` bigger on hover, you can use the following CSS code:

```css
#PathCollection_1 use:hover {
  transform: scale(1.2);  /* Adjust the scale factor as needed */
  transition: transform 0.3s;  /* Smooth transition effect */
}
```

This CSS will scale up the `<use>` elements when hovered over, providing a smooth transition effect. Adjust the scale factor to achieve the desired size.


In [134]:
answer = chat("""
Where would I place this style tag in my HTML?
""")
print(answer)

You can place the `<style>` tag within the `<head>` section of your HTML document. Here’s an example of how you can include it:

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Your Page Title</title>
  <style>
    #PathCollection_1 use:hover {
      transform: scale(1.2); /* Adjust the scale factor as needed */
      transition: transform 0.3s; /* Smooth transition effect */
    }
  </style>
</head>
<body>
  <!-- Your HTML content goes here -->
</body>
</html>
```

Alternatively, if you are using an external CSS file, you could add the CSS rule to that file instead. Just make sure to link the external CSS file within the `<head>` section of your HTML:

```html
<head>
  <link rel="stylesheet" href="styles.css">
</head>
```

And then in `styles.css` file:

```css
#PathCollection_1 use:hover {
  transform: scale(1.2); /* Adjust the scale factor as needed */
  transition: transform 0

In [135]:
chat.display()

###USER

I'd like to Make "use" HTML tags that are sub-elements of id "PathCollection_1" a bit bigger on hover. Brief response, please.

***
###ASSISTANT
To make the `<use>` elements within the element with the id `PathCollection_1` bigger on hover, you can use the following CSS code:

```css
#PathCollection_1 use:hover {
  transform: scale(1.2);  /* Adjust the scale factor as needed */
  transition: transform 0.3s;  /* Smooth transition effect */
}
```

This CSS will scale up the `<use>` elements when hovered over, providing a smooth transition effect. Adjust the scale factor to achieve the desired size.
***
###USER

Where would I place this style tag in my HTML?

***
###ASSISTANT
You can place the `<style>` tag within the `<head>` section of your HTML document. Here’s an example of how you can include it:

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Your Page Title</title>
  <style>
    #PathCollection_1 use:hover {
      transform: scale(1.2); /* Adjust the scale factor as needed */
      transition: transform 0.3s; /* Smooth transition effect */
    }
  </style>
</head>
<body>
  <!-- Your HTML content goes here -->
</body>
</html>
```

Alternatively, if you are using an external CSS file, you could add the CSS rule to that file instead. Just make sure to link the external CSS file within the `<head>` section of your HTML:

```html
<head>
  <link rel="stylesheet" href="styles.css">
</head>
```

And then in `styles.css` file:

```css
#PathCollection_1 use:hover {
  transform: scale(1.2); /* Adjust the scale factor as needed */
  transition: transform 0.3s; /* Smooth transition effect */
}
```

Placing the CSS in the `<head>` section or an external CSS file ensures that the styles are loaded before the HTML content is rendered.

#### Doctor's assistant (careful!)

In [136]:
p_sys = """You are an expert doctor's assistant focused on enhancing individuals' overall wellness and physical performance beyond just treating illness.
You will aid doctors in finding possible causes and treatment options for patiens' issues.

Input: Summary of a patient's issue
Output: A list of underlying causes, each with a connection to symptoms and its potential solution, for the doctor to evaluate.
"""

In [137]:
# qa = gen_qa_sys(p_sys)
chat = NewChat(p_sys, model="gpt-4o-mini")

In [138]:
p_user = """
- 35 yo male, no history of illness.
- Patient reports letargy in the morning, even after getting enough sleep, more often than not.
- Muscle weakness in the first minutes to hours of the day, most noticeably lack of grip strength.
- Hydration and coffee often help, but patient feels like they are not sufficient to get him to a normal state.
"""

answer = chat(p_user)
print(answer)

### Potential Underlying Causes and Solutions

#### 1. **Sleep Apnea**
   - **Connection to Symptoms:** Sleep apnea can disrupt sleep quality without the patient being aware, leading to morning lethargy and muscle weakness.
   - **Potential Solution:** 
     - Conduct a sleep study (polysomnography) to diagnose sleep apnea.
     - If diagnosed, CPAP (Continuous Positive Airway Pressure) therapy could be recommended.
     - Lifestyle changes such as weight management, sleeping position changes, and avoiding alcohol before bed.

#### 2. **Thyroid Dysfunction (Hypothyroidism)**
   - **Connection to Symptoms:** Hypothyroidism can cause fatigue and muscle weakness, especially in the mornings.
   - **Potential Solution:**
     - Assess thyroid function through blood tests (TSH, T3, T4 levels).
     - If hypothyroidism is confirmed, thyroid hormone replacement therapy (e.g., levothyroxine) could be prescribed.

#### 3. **Nutritional Deficiencies (e.g., Vitamin D, B12)**
   - **Connection to S

In [139]:
answer = chat('How about magnesium defficiency?')
print(answer)

### Magnesium Deficiency

#### **Connection to Symptoms:**
   - **Morning Lethargy:** Magnesium plays a critical role in energy production and regulation of sleep patterns. A deficiency can impair sleep quality, leading to morning lethargy.
   - **Muscle Weakness:** Magnesium is essential for muscle function and relaxation. Low levels can result in muscle cramps, spasms, or weakness, affecting grip strength especially in the morning.

#### **Potential Solution:**
   - **Diagnostic Testing:** Measure serum magnesium levels through a blood test to confirm deficiency.
   - **Dietary Adjustments:** Increase consumption of magnesium-rich foods such as leafy green vegetables, nuts, seeds, whole grains, and legumes.
   - **Supplementation:** If dietary adjustments are insufficient, consider magnesium supplements after consulting with the doctor to determine the appropriate dosage.
   - **Hydration:** Continue maintaining good hydration, as dehydration can exacerbate magnesium deficiency sympt

In [140]:
chat.display()

###SYSTEM
You are an expert doctor's assistant focused on enhancing individuals' overall wellness and physical performance beyond just treating illness.
You will aid doctors in finding possible causes and treatment options for patiens' issues.

Input: Summary of a patient's issue
Output: A list of underlying causes, each with a connection to symptoms and its potential solution, for the doctor to evaluate.

***
###USER

- 35 yo male, no history of illness.
- Patient reports letargy in the morning, even after getting enough sleep, more often than not.
- Muscle weakness in the first minutes to hours of the day, most noticeably lack of grip strength.
- Hydration and coffee often help, but patient feels like they are not sufficient to get him to a normal state.

***
###ASSISTANT
### Potential Underlying Causes and Solutions

#### 1. **Sleep Apnea**
   - **Connection to Symptoms:** Sleep apnea can disrupt sleep quality without the patient being aware, leading to morning lethargy and muscle weakness.
   - **Potential Solution:** 
     - Conduct a sleep study (polysomnography) to diagnose sleep apnea.
     - If diagnosed, CPAP (Continuous Positive Airway Pressure) therapy could be recommended.
     - Lifestyle changes such as weight management, sleeping position changes, and avoiding alcohol before bed.

#### 2. **Thyroid Dysfunction (Hypothyroidism)**
   - **Connection to Symptoms:** Hypothyroidism can cause fatigue and muscle weakness, especially in the mornings.
   - **Potential Solution:**
     - Assess thyroid function through blood tests (TSH, T3, T4 levels).
     - If hypothyroidism is confirmed, thyroid hormone replacement therapy (e.g., levothyroxine) could be prescribed.

#### 3. **Nutritional Deficiencies (e.g., Vitamin D, B12)**
   - **Connection to Symptoms:** Low levels of certain vitamins and minerals can lead to fatigue and muscle weakness.
   - **Potential Solution:**
     - Check vitamin and mineral levels through blood tests (e.g., Vitamin D, B12, Iron).
     - Dietary adjustments or supplementation based on deficiency findings.

#### 4. **Adrenal Insufficiency or Dysfunction**
   - **Connection to Symptoms:** Issues with adrenal function can lead to inadequate cortisol production, resulting in morning fatigue and muscle weakness.
   - **Potential Solution:**
     - Test cortisol levels (morning cortisol test) and possibly ACTH stimulation test.
     - If adrenal insufficiency is diagnosed, hormone replacement therapy may be necessary.

#### 5. **Diabetes or Blood Sugar Irregularities**
   - **Connection to Symptoms:** Blood sugar fluctuations can cause morning fatigue and muscle weakness.
   - **Potential Solution:**
     - Monitor blood glucose levels (fasting glucose, HbA1c).
     - Diet modification, increased physical activity, and possibly medication if blood sugar levels are abnormal.

#### 6. **Chronic Fatigue Syndrome (CFS)**
   - **Connection to Symptoms:** CFS can lead to persistent fatigue that is not relieved by sleep and muscle weakness.
   - **Potential Solution:**
     - A thorough evaluation to rule out other conditions.
     - Symptomatic treatment which may include graded exercise therapy (GET), cognitive behavioral therapy (CBT), and possibly medication for symptom management.

#### 7. **Electrolyte Imbalance**
   - **Connection to Symptoms:** Electrolyte imbalances can contribute to fatigue and muscle weakness.
   - **Potential Solution:**
     - Check serum electrolyte levels (e.g., sodium, potassium, calcium, magnesium).
     - Correct any imbalances with appropriate dietary changes or supplements.

#### 8. **Chronic Stress or Depression**
   - **Connection to Symptoms:** Chronic stress or depression can affect energy levels and physical strength.
   - **Potential Solution:**
     - Screening for stress and depressive symptoms.
     - Interventions such as counseling, lifestyle changes, or medication (antidepressants or anxiolytics) as appropriate.

#### 9. **Medications or Substance Use**
   - **Connection to Symptoms:** Certain medications or excessive use of substances (including caffeine) can cause morning lethargy and muscle weakness.
   - **Potential Solution:**
     - Review the patient's medication and substance use history.
     - Adjust or change medications if needed, while educating on optimal caffeine consumption.

---

These potential underlying causes and solutions should be further evaluated through patient history, physical examination, and appropriate diagnostic testing. Each solution provides a pathway for targeted treatment to help alleviate the patient's symptoms.
***
###USER
How about magnesium defficiency?
***
###ASSISTANT
### Magnesium Deficiency

#### **Connection to Symptoms:**
   - **Morning Lethargy:** Magnesium plays a critical role in energy production and regulation of sleep patterns. A deficiency can impair sleep quality, leading to morning lethargy.
   - **Muscle Weakness:** Magnesium is essential for muscle function and relaxation. Low levels can result in muscle cramps, spasms, or weakness, affecting grip strength especially in the morning.

#### **Potential Solution:**
   - **Diagnostic Testing:** Measure serum magnesium levels through a blood test to confirm deficiency.
   - **Dietary Adjustments:** Increase consumption of magnesium-rich foods such as leafy green vegetables, nuts, seeds, whole grains, and legumes.
   - **Supplementation:** If dietary adjustments are insufficient, consider magnesium supplements after consulting with the doctor to determine the appropriate dosage.
   - **Hydration:** Continue maintaining good hydration, as dehydration can exacerbate magnesium deficiency symptoms.
   - **Overall Lifestyle:** Encourage a balanced diet, regular exercise, and good sleep hygiene to support overall health and magnesium levels.

Incorporating the evaluation and treatment of potential magnesium deficiency could be a valuable addition to the patient's care plan, alongside the other possible causes and solutions listed previously.

In [141]:
answer = chat.retry()
print(answer)

### Potential Underlying Cause: Magnesium Deficiency

#### Connection to Symptoms:
Magnesium is an essential mineral involved in numerous biochemical reactions in the body, including those that regulate muscle and nerve function, blood glucose control, and energy production. Deficiency in magnesium can result in symptoms such as:

- **Lethargy**: Magnesium plays a crucial role in energy production. A deficiency might lead to chronic fatigue and low energy levels, particularly noticeable in the mornings.
- **Muscle Weakness**: Magnesium is vital for muscle function. Insufficient magnesium levels can lead to muscle cramps, weakness, and poor muscle control, including decreased grip strength.

#### Potential Solution:
- **Diagnostic Testing**: 
  - Blood tests to measure magnesium levels (serum magnesium test).
  
- **Dietary Adjustments**:
  - Increasing intake of magnesium-rich foods such as nuts, seeds, whole grains, green leafy vegetables, legumes, and bananas.
  
- **Supplementation*

#### Sys prompt

In [66]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."
question = "What is money?"

c = ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": question}])

- [Model options](https://platform.openai.com/docs/models)

In [67]:
def response(c):
    if type(c) == openai.types.chat.chat_completion.ChatCompletion:
        return c.choices[0].message.content
    elif type(c) == openai.types.completion.Completion:
        return c.choices[0].text
    else: raise TypeError(f"input should be of type <Completion> or <ChatCompletion>, but it is or type {type(c)}")

In [68]:
response(c)

"Money is like bloody gold, mate. It's like the cold hard cash that makes the world go round. It's what ya use to buy stuff, pay your bills, and live your best life. Without money, well, you'd be up shit creek without a paddle, that's for sure."

#### Usage

In [69]:
print(c.usage)

CompletionUsage(completion_tokens=61, prompt_tokens=31, total_tokens=92, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))


In [70]:
0.002 / 1000 * 150  # GPT 3.5

0.0003

In [71]:
0.03 / 1000 * 150  # GPT 4

0.0045

#### askgpt

In [154]:
c = ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [155]:
type(c)

openai.types.chat.chat_completion.ChatCompletion

In [156]:
response(c)

"Absolutely, mate! Think of money as a bit of a kangaroo in the Aussie bush. It's a handy tool that helps you hop around and get what you need. Just like how kangaroos bounce from place to place, money allows you to jump into trades and buy goods and services.\n\nBut not all kangaroos are created equal, right? Some are big and strong, while others are smaller and quicker. Similarly, you've got different types of money—cash, coins, digital dollars, and so on. They all serve a purpose, but you've gotta know how to use them.\n\nAnd just like you wouldn't want to get caught in a mob of kangaroos without a plan, you don't want to be caught short on cash when you need to pay for something important! So, in the end, money is a means to an end, a tool to help you navigate life, whether you're buying a meat pie or saving up for a trip to the Great Barrier Reef."

In [157]:
def askgpt(user, system=None, model="gpt-4o-mini", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [158]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

"Oh mate, that's a real corker of a question! The meaning of life is like a boomerang - it can come back to ya in all different ways. Some reckon it's all about finding happiness and fulfillment, while others say it's about making a positive impact on the world. At the end of the day, it's up to each of us to figure out what gives our life meaning and purpose. It's like trying to catch a wave - you gotta ride it and make the most of it while you can, ya know?"

- [Limits](https://platform.openai.com/docs/guides/rate-limits/what-are-the-rate-limits-for-our-api)

Created by Bing:

In [159]:
def call_api(prompt, model="gpt-4o-mini"):
    msgs = [{"role": "user", "content": prompt}]
    try: return ChatCompletion.create(model=model, messages=msgs)
    except client.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [160]:
response(call_api("What's the world's funniest joke? Has there ever been any scientific analysis?"))

'It is difficult to determine the funniest joke in the world as humor is subjective and varies greatly among cultures and individuals. However, there have been several studies and experiments conducted to try to identify the world\'s funniest joke.\n\nOne such study was conducted by British psychologist Richard Wiseman in 2002, where he asked participants to rate jokes from various categories and styles. The joke that was ultimately rated the funniest in his study was:\n\n"Two hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing and his eyes are glazed. The other guy whips out his phone and calls the emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says \'Calm down. I can help. First, let\'s make sure he\'s dead.\' There is a silence, then a shot is heard. Back on the phone, the guy says \'OK, now what?\'"\n\nWhile this joke may have been rated the funniest in Wiseman\'s study, humor is still highly subjective and wha

In [161]:
c = Completion.create(
    prompt="Australian Jeremy Howard is ",
    model="gpt-3.5-turbo-instruct", echo=True)

In [162]:
response(c)

'Australian Jeremy Howard is 1 of the co-founders of Fast.AI, which is committed to creating deep'

In [163]:
story = '''I wake up this morning from a strange dream of cold '''

In [164]:
c = Completion.create(
    prompt=story,
    model="gpt-3.5-turbo-instruct", echo=False, max_tokens=30)

In [165]:
print('...', response(c))

... war at the brink of another world war
The dream felt so real, it sent shivers down my spine
As I look around my room and


## OpenAI API with code interpreter

In [84]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

#### Example with sums function

In [85]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [86]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [87]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [88]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [89]:
c

ChatCompletion(id='chatcmpl-A7gyj5SHeL41HQdqc25KU45J1UeD4', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"a":6,"b":3}', name='sums'), tool_calls=None))], created=1726396869, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=96, total_tokens=114, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [90]:
m = c.choices[0].message
m

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"a":6,"b":3}', name='sums'), tool_calls=None)

In [91]:
k = m.function_call.arguments
print(k)

{"a":6,"b":3}


In [92]:
funcs_ok = {'sums', 'python'}

In [93]:
c.choices[0].message.function_call.arguments

'{"a":6,"b":3}'

In [94]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    try:
        return f(**json.loads(fc.arguments))
    except ValueError:
        return f(fc.arguments)

In [95]:
call_func(c)

9

In [96]:
import ast

def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [97]:
run("""
a=1
b=2
a+b
""")

3

In [98]:
def python(code:str, safe_mode:bool=False):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    if safe_mode:
        go = input(f'Proceed with execution?\n```\n{code}\n```\n')
        if go.lower()!='y': return '#FAIL#'
    return run(code)

In [99]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [100]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    if 'function_call' not in dir(c.choices[0].message):
        print(txt_out)
        return  # No code output
    code = c.choices[0].message.function_call.arguments
    if code[0] == '{': code = json.loads(code)['code']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    result = run(code)
    print(txt_out)
    return result

In [101]:
call_func(c)

479001600

In [102]:
code_response(c)


import math
factorial = math.factorial(12)
factorial
>>> 


479001600

#### Using Python's exec & eval

In [103]:
exec_schema = {
    'name': 'exec',
    'description': 'Execute the given source Python code',
    'parameters': {
        'title': 'Input for `exec`',
        'type': 'object',
        'properties': {'source': {'title': 'S', 'type': 'string'}},
        'required': ['source']}}

In [104]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    msg = c.choices[0].message
    if 'function_call' not in dir(msg) or msg.function_call is None:
        print(txt_out)
        return  # No code output
    code = msg.function_call.arguments
    if code[0] == '{': code = json.loads(code)['source']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    if repl:
        code_body = '\n'.join(code.split('\n')[:-1])
        code_footer = code.split('\n')[-1]
        exec(code_body, locals())
        result = eval(code_footer, locals())
        txt_out += str(result)
    else:
        exec(code, locals())
        result = None
    print(txt_out)
    return result

In [105]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [106]:
factorial_result = code_response(c, repl=True)


import math
factorial = math.factorial(12)
factorial
>>> 479001600


In [107]:
factorial_result

479001600

In [108]:
def ask_python(s):
    c = askgpt(s,
           system = "Use python for any required computations.",
           functions=[exec_schema])
    result = code_response(c, repl=True)
    return result

In [109]:
prime_question = "What is the largest prime less than 434?"
prime_result = ask_python(prime_question)


import sympy

largest_prime = sympy.prevprime(434)
largest_prime
>>> 433


###### Using result in later calls

In [110]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[exec_schema],
    messages=[{"role": "user", "content": prime_question},
              {"role": "function", "name": "exec", "content": str(prime_result)}])

In [111]:
print(response(c))

The largest prime number less than 434 is 433.


###### And we didn't break its basic use!

In [112]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [113]:
code_response(c)

The capital of France is Paris.


## Prediction from current execution!

Comments are not saved in history_manager!!! We have to use strings at the start of the cell.

In [ ]:
def get_history(): return get_ipython().history_manager.input_hist_raw
def clear_history(): get_ipython().history_manager.input_hist_raw = []

In [ ]:
def complete_cell():
    _func_call = '\n' + complete_cell.__name__ + '()'
    raw = get_history()
    nb = '\n\n\n#! CELL\n\n'.join(raw).split(_func_call)[0]
    c = Completion.create(prompt=nb,
                      model="gpt-3.5-turbo-instruct",
                      echo=False,
                      best_of=1,
                      stop= "#! CELL")
    return response(c).strip()

In [ ]:
clear_history()

In [ ]:
'''Characters'''
'abcdefghijklmnopqrstuvwxyz'

'abcdefghijklmnopqrstuvwxyz'

In [ ]:
'''Chinese characters'''
print(complete_cell())

中国字


## Open source models

In [3]:
!pip install transformers -qq
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams

INFO 09-15 10:08:08 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


- [HF leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [fasteval](https://fasteval.github.io/FastEval/)

In [4]:
# mn = "meta-llama/Llama-2-7b-hf"
mn = 'neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16'

In [5]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
toks

{'input_ids': tensor([[76665, 20462,   374,   264,   220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [7]:
tokr.batch_decode(toks['input_ids'])

['Jeremy Howard is a ']

In [11]:
model_id = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16"
llm = LLM(
    model=model_id,
    tensor_parallel_size=1,
    max_model_len=8192
)
sampling_params = SamplingParams(temperature=0.6, top_p=0.9, max_tokens=256)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [13]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
outputs = llm.generate(prompts, sampling_params)
generated_text = outputs[0].outputs[0].text
print(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it, est. speed input: 17.77 toks/s, output: 44.41 toks/s]



Arrr, ye be askin' who I be? Well, matey, I be a swashbucklin' chatbot, a scurvy dog with a penchant fer spoutin' pirate speak! Me name be Captain Chat, and I be here to guide ye through the seven seas o' conversation. What be bringin' ye to these waters today, eh?


In [ ]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.int4)

In [ ]:
# %%time
# res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
# res

In [ ]:
# tokr.batch_decode(res)

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
# %%time
# res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
# res

In [ ]:
mn = 'TheBloke/Llama-2-7b-Chat-GPTQ'
# mn = 'neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
# mn = 'TheBloke/Llama-2-13B-GPTQ'
# model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
# %%time
# res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
# res

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 50)

#### [StableBeluga-7B](https://huggingface.co/stabilityai/StableBeluga-7B)

In [ ]:
del model

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys): return f"### System:\n{syst}\n### User:\n{user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Jeremy Howard?"

In [ ]:
gen(mk_prompt(ques), 150)

[OpenOrca/Platypus 2](https://huggingface.co/Open-Orca/OpenOrca-Platypus2-13B)

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques), 150)

#### Retrieval augmentation

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install wikipedia-api
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
print(jh_page[:500])

In [ ]:
len(jh_page.split())

In [ ]:
def mk_prompt_context(question, context):
    return f"""Answer the question with the help of the provided context.\n\n## Context\n\n{context}\n\n## Question\n\n{question}## Answer\n"""

In [ ]:
res = gen(mk_prompt_context(ques, jh_page), 300)

In [ ]:
print(res[0].split('## Answer\n')[1])

In [ ]:
!pip install sentence-transformers -qq
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
tb_page = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [ ]:
tb = tb_page.split('\n\n')[0]
print(tb[:380])

In [ ]:
q_emb,jh_emb,tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [ ]:
tb_emb.shape

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

### Private GPTs

- [Sooo many](https://github.com/h2oai/h2ogpt/blob/main/docs/README_LangChain.md#what-is-h2ogpts-langchain-integration-like)

## Fine tuning

In [ ]:
import datasets

[knowrohit07/know_sql](https://huggingface.co/datasets/knowrohit07/know_sql)

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

In [ ]:
ds

In [ ]:
trn = ds['train']
trn[3]

`accelerate launch -m axolotl.cli.train sql.yml`

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

In [ ]:
print(sql_prompt(tst))

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
ax_model = '/home/jhoward/git/ext/axolotl/qlora-out'

In [ ]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [ ]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',
                                             torch_dtype=torch.bfloat16, device_map=0)

# TODO: Get config to Drive
model = PeftModel.from_pretrained(model, ax_model)
model = model.merge_and_unload()
model.save_pretrained('sql-model')

In [ ]:
toks = tokr(sql_prompt(tst), return_tensors="pt")

In [ ]:
res = model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')

In [ ]:
print(tokr.batch_decode(res)[0])

## [llama.cpp](https://github.com/abetlen/llama-cpp-python)

[TheBloke/Llama-2-7b-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF)

In [ ]:
!pip install llama_cpp_python -qq
from llama_cpp import Llama

In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf

In [ ]:
llm = Llama(model_path="/content/llama-2-7b-chat.Q4_K_M.gguf")

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

In [ ]:
print(output['choices'])